<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Estimating-the-Parameters-from-the-given-Data" data-toc-modified-id="Estimating-the-Parameters-from-the-given-Data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Estimating the Parameters from the given Data</a></span><ul class="toc-item"><li><span><a href="#Loading-the-Data" data-toc-modified-id="Loading-the-Data-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Loading the Data</a></span></li><li><span><a href="#MLE-method" data-toc-modified-id="MLE-method-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>MLE method</a></span><ul class="toc-item"><li><span><a href="#Underlying-Math" data-toc-modified-id="Underlying-Math-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Underlying Math</a></span></li><li><span><a href="#Math-to-code" data-toc-modified-id="Math-to-code-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Math to code</a></span></li></ul></li><li><span><a href="#Method-of-Moments" data-toc-modified-id="Method-of-Moments-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Method of Moments</a></span><ul class="toc-item"><li><span><a href="#Underlying-Math" data-toc-modified-id="Underlying-Math-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Underlying Math</a></span></li><li><span><a href="#Math-to-code" data-toc-modified-id="Math-to-code-1.3.2"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>Math to code</a></span></li></ul></li><li><span><a href="#Bootstrap-Estimation-Method" data-toc-modified-id="Bootstrap-Estimation-Method-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Bootstrap Estimation Method</a></span></li><li><span><a href="#Results" data-toc-modified-id="Results-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Results</a></span></li></ul></li><li><span><a href="#New-Random-Variables" data-toc-modified-id="New-Random-Variables-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>New Random Variables</a></span><ul class="toc-item"><li><span><a href="#Estimation-of-parameters-directly-from-the-dataset" data-toc-modified-id="Estimation-of-parameters-directly-from-the-dataset-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Estimation of parameters directly from the dataset</a></span></li></ul></li></ul></div>

# Estimating the Parameters from the given Data

## Loading the Data

In [1]:
#Importing the libraries to be used.
import pandas as pd
import numpy as np
import random

In [2]:
#Utilities to be used.
sqrt = lambda r: r**(0.5)
mean = lambda lst: sum(lst)/len(lst)   #Sample Mean
var = lambda lst,mu: sum([(x-mu)**2 for x in lst])/(len(lst)-1)   #Sample Variance
style_df = lambda df: df.head(5).style.set_table_styles(
    [{
        'selector':
        'th',
        'props': [('background', '#FFFEE3'), ('color', 'black'),
                  ('font-family', 'verdana')]
    }, {
        'selector': 'td',
        'props': [('font-family', 'verdana')]
    }, {
        'selector': 'tr:nth-of-type(odd)',
        'props': [('background', '#ADD8E6')]
    }, {
        'selector': 'tr:nth-of-type(even)',
        'props': [('background', 'white')]
    }, {
        'selector': 'tr:hover',
        'props': [('background-color', '#FFFEE3')]
    }])
pd.set_option('max_rows', None)


In [3]:
data_frame = pd.read_csv(r'Q4.csv')
data_frame.dropna()
style_df(data_frame)

,X,Y
0,11.046971,17.451000
1,7.931476,10.458889
2,10.489960,16.313118
3,10.993082,6.999862
4,9.485375,17.917017


In [4]:
X = data_frame['X'].tolist()
Y = data_frame['Y'].tolist()

## MLE method

### Underlying Math

* Let us assume that a random variable X has a Gaussian distribution whose parameters $\mu$ and $\sigma$ are unknown to us.
* Now, consider $n-$points drawn *i.i.d.* from such a distribution.
* It is reasonable to assume that the parameters that maximizes the likelihood of observing the $n-$points drawn above are the actual parameters of the underlying distribution. For $\Theta = \{\mu, \sigma\}$ we have,
$$\Theta_{MLE} = argmax_{\Theta}  P(X = x_1,X = x_2,\dots,X = x_n ; \Theta) $$
* Since the samples are *i.i.d.*,
$$\Theta_{MLE} =argmax_{\Theta} \prod_{i=1}^{n} P(X = x_i ; \Theta) $$
* We can apply $log$ to likelihood and since it is an monotonically increasing function the estimates remain the same,
$$\Theta_{MLE} =argmax_{\Theta} \sum_{i=1}^{n} P(X = x_i ; \Theta)$$
* Substituting the **PDF** of Normal Distribution we get,
$$\Theta_{MLE} =argmax_{\Theta}\sum_{i=1}^{n} \left[ -\frac{1}{2}\log{2\pi} - \log{\sigma} - \frac{(x_i - \mu)^2}{2\sigma^2}\right]$$ 
* Maximizing w.r.t. $\mu$ we get,
$$\mu_{MLE} = \frac{1}{n} \sum_{i=1}^{n} x_i$$
* Similarly we could also maximise w.r.t. $\sigma$,
$$\sigma_{MLE} = \sqrt{\frac{1}{n}\sum_{i=1}^{n}(x_i - \mu_{MLE})^2}$$

### Math to code

In [5]:
#A class that given a list of observed data points estimates the parameters of the underlying dist using MLE.
class Max_Lklhd_Estimator:
    def __init__(self, ob_data, dist):
        self.X = ob_data
        self.dist = dist
        self.params = {}

    def get_params(self):
        if self.dist == 'Gaussian':  #Assumption that the underlying dist is Gaussian.
            n = len(self.X)
            mu = mean(self.X)
            self.params['Mean'] = mu
            rad = sum([(x - mu)**2 for x in self.X]) / n
            self.params['Std'] = sqrt(rad)
        else:
            pass
        return self.params

## Method of Moments
### Underlying Math

* $\mathbb{E}[X^k]$ represents the $k-th$ theoretical moment of the distribution of the distribution of $X$ about the origin.
* The $k-th$ sample (n-samples) moment ($\mathbb{M}_{k}$) can be calculated using, $$\mathbb{M}_{k} = \frac{1}{n}\sum_{i=1}^{n} X_{i}^{k} $$
* In Methods of Moments, both of these moments are assumed to be the same.
* Therefore, mean of the distribution is equal to the sample mean.
* Standard Deviation of the distributor can be calculated using,
\begin{equation*}
\sigma = \sqrt{\mathbb{E}[X^2] - \mathbb{E}[X]^{2}}= \sqrt{\mathbb{M}_{2} - \mathbb{M}_{1}^{2}}
\end{equation*}
* In case of a Gaussian r.v., both MLE and MoM yield the same results.

### Math to code

In [6]:
class MoM_Estimator:
    def __init__(self, ob_data, num_moments, dist):
        self.X = ob_data
        self.dist = dist
        self.no_mom = num_moments + 1  #Number of moments to be estimated.
        self.params = {}

    def get_moments(self):
        N = len(self.X)
        moments = {}
        for moment in range(1, self.no_mom):
            moments[str(moment)] = sum([x**moment for x in self.X]) / N
        return moments

    def get_params(self):
        Moments = self.get_moments()
        if self.dist == 'Gaussian':
            M_1 = Moments['1']
            M_2 = Moments['2']
            self.params['Mean'] = M_1
            self.params['Std'] = sqrt(M_2 - (M_1**2))
        else:
            pass
        return self.params

## Bootstrap Estimation Method
* This is a non-parametric method that employs Monte Carlo technique to approximate the sampling distribution. 
* We draw $S$ samples of size $N$ from the the original dataset $\mathcal{D}$ to use in the Monte-Carlo algorithm.
* Let $\mathcal{D} = \{x_1,x_2,\dots,x_n\}$. In each bootstrap step, we draw samples with replacemnt from $\mathcal{D}$ until we have another set of size $n$.
* The probabilty of choosing any sample in a draw is $\frac{1}{n}$ and consquently the probability of not choosing is $1 - \frac{1}{n}$.
* Therfore the probabilty of a sample being not selcted in any one of the n-draws is,
$$P = \left(1 - \frac{1}{n}\right)^n $$
* For large n, 
$$\lim_{n \to \infty}P =\lim_{n \to \infty} \left(1 - \frac{1}{n}\right)^n  = \frac{1}{e}$$
* Hence the average number of samples being used in each step of bootstrap is $\left(1-\frac{1}{e}\right) \approx 0.632$ of the total samples.

In [7]:
class Bootstrap_Estimator():
    def __init__(self, ob_data, num_bootstrap_steps):
        self.X = ob_data
        self.B = num_bootstrap_steps
        self.params = {}

    def get_bootstrap_sample(self, X, S):
        #A function that chooses with replacement a sample dataset of size S from D.
        bootstrap_sample = [random.choice(X) for _ in self.X]
        return bootstrap_sample

    def get_params(self):
        n = len(self.X)
        sample_means = []
        sample_std = []
        random.seed(123)
        for _ in range(self.B):
            X_boot_sample = self.get_bootstrap_sample(self.X, n)
            sample_means.append(mean(X_boot_sample))
            sample_std.append(sqrt(var(X_boot_sample, sample_means[-1])))
        self.params['Mean'] = mean(sample_means)
        self.params['Std'] = mean(sample_std)
        return self.params

## Results

In [8]:
class Frequentist_Estimators(Max_Lklhd_Estimator, MoM_Estimator,
                             Bootstrap_Estimator):
    def __init__(self, dataset, specs):
        self.names = [key for key in dataset.keys()]
        self.data = [val for k, val in dataset.items()]
        self.specs = specs
        self.params = {}

    def get_all_params(self):
        D, N_m, N_b = self.specs  #D -> Type of dist, N_m->no of moments,N_b->no of bootsteps
        for X, name in zip(self.data, self.names):
            MLE = Max_Lklhd_Estimator(X, D)
            MoM = MoM_Estimator(X, N_m, D)
            Bootstrap = Bootstrap_Estimator(X, N_b)
            for est_inst, estimator in zip([MLE, MoM, Bootstrap],
                                           ['MLE', 'MoM', 'Bootstrap']):
                self.params[name + '_' + estimator +
                            '_estimate'] = est_inst.get_params()
        return self.params

In [9]:
dataset = {'X': X, 'Y': Y}  #X -> Dataset 1. Y -> Dataset 2.
specs = ['Gaussian', 2, 100]
ests_inst = Frequentist_Estimators(dataset, specs)
param_df = pd.DataFrame(ests_inst.get_all_params())
style_df(param_df)

,X_MLE_estimate,X_MoM_estimate,X_Bootstrap_estimate,Y_MLE_estimate,Y_MoM_estimate,Y_Bootstrap_estimate
Mean,9.991867,9.991867,9.988125,14.812310,14.812310,14.809416
Std,1.974909,1.974909,1.973210,4.908668,4.908668,4.913657


# New Random Variables

## Estimation of parameters directly from the dataset

* The new random variables are defined by,
\begin{align*}
W &= min\{X,Y\}\\
V &=max\{X,Y\}\\
A &= |X-Y| \\
B &= |X| - |Y|
\end{align*}

In [10]:
data_frame['W'] = data_frame[['X', 'Y']].min(axis=1)
data_frame['V'] = data_frame[['X', 'Y']].max(axis=1)
data_frame['A'] = data_frame['V'] - data_frame['W']
data_frame['B'] = data_frame['X'].abs() - data_frame['Y'].abs()
style_df(data_frame)

,X,Y,W,V,A,B
0,11.046971,17.451000,11.046971,17.451000,6.404028,-6.404028
1,7.931476,10.458889,7.931476,10.458889,2.527414,-2.527414
2,10.489960,16.313118,10.489960,16.313118,5.823158,-5.823158
3,10.993082,6.999862,6.999862,10.993082,3.993220,3.993220
4,9.485375,17.917017,9.485375,17.917017,8.431642,-8.431642


In [11]:
dataset = {}
[V, W, A, B] = [data_frame[col].tolist() for col in ['V', 'W', 'A', 'B']]
for key, val in zip(['V', 'W', 'A', 'B'], [V, W, A, B]):
    dataset[key] = val

In [12]:
specs = ['Gaussian', 2, 100]
ests_inst = Frequentist_Estimators(dataset, specs)
param_df = pd.DataFrame(ests_inst.get_all_params())
style_df(param_df)

,V_MLE_estimate,V_MoM_estimate,V_Bootstrap_estimate,W_MLE_estimate,W_MoM_estimate,W_Bootstrap_estimate,A_MLE_estimate,A_MoM_estimate,A_Bootstrap_estimate,B_MLE_estimate,B_MoM_estimate,B_Bootstrap_estimate
Mean,15.341197,15.341197,15.333868,9.462981,9.462981,9.463672,5.878216,5.878216,5.870196,-4.836710,-4.836710,-4.835973
Std,4.167737,4.167737,4.184437,2.228650,2.228650,2.218522,4.207327,4.207327,4.218908,5.339269,5.339269,5.343136
